<a href="https://colab.research.google.com/github/tapilab/text-analytics-tutorial/blob/master/TextAnalyticsTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Analytics Tutorial
**Aron Culotta**  
**Associate Professor of Computer Science**  
**Illinois Institute of Technology**  
*November 13, 2019*

## Survey of text analysis problems

First, let's look at an overview of a range of text analysis tasks.

Now, we'll look in more depth at two common text analysis tasks: classification and clustering.

## Classification

### Read data

In [8]:
# @title Load the keras models
print('hi there')

hi there


### Preprocessing

### Feature engineering

### Training

### Evaluation

### Error analysis

# Clustering